# EXTRACT 

In [1]:
import awswrangler as awr
import logging
import pandas as pd
import os

logging.basicConfig(
    level=logging.INFO,  # Exibe mensagens a partir de INFO
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.StreamHandler()  # Garante logs no console
    ]
)

logging.info('\n ----------------------------------------------------------------------------------')
logging.info('\n Executando Rotina: Movimentação de Placas')

class Extract:

    def __init__(self):
        self.path = r"C:\Users\raphael.almeida\Documents\Processos\placas_movimentacoes"

    def extract_all_ativacoes(self):

        try:
            dir_query = os.path.join(self.path, 'sql', 'all_boards_ATIVOS.sql')
            with open(dir_query, 'r') as file:
                query = file.read()
            df_ativacoes = awr.athena.read_sql_query(query, database='silver')
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info('\n Consulta de ativações extraída com sucesso!')
            return df_ativacoes
        except Exception as e:
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'\n Falha ao extrair a consulta de ativações: {e}')
            return None

    def extract_all_cancelamentos(self):

        try:
            dir_query = os.path.join(self.path, 'sql', 'all_boards_CANCELADOS.sql')
            with open(dir_query, 'r') as file:
                query = file.read()
            df_cancelamentos = awr.athena.read_sql_query(query, database='silver')
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info('\n Consulta de cancelamentos extraída com sucesso!')
            return df_cancelamentos
        except Exception as e:
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'\n Falha ao extrair a consulta de cancelamentos: {e}')
            return None

extract = Extract()
df_ativacoes = extract.extract_all_ativacoes()
df_cancelamentos = extract.extract_all_cancelamentos()



2025-08-26 11:23:10,188 - INFO - 
 ----------------------------------------------------------------------------------
2025-08-26 11:23:10,188 - INFO - 
 Executando Rotina: Movimentação de Placas
2025-08-26 11:23:10,221 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 11:23:12,225 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 11:23:13,710 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 11:23:16,171 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 11:23:18,358 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 11:23:20,347 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 11:23:22,240 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 11:23:25,193 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 11:23:27,1

# TRANSFORM

In [ ]:
# IMPORTANDO MÓDULOS E PACOTES
import pandas as pd
import numpy as np
import datetime as dt
import logging
import traceback
import numpy as np

class Transform:
        
    def __init__(self) -> None:
        pass


    def transforming_files(self):

        try:

            # INICIALIANDO DATAFRAMES VAZIOS
            df_final_ativacoes = pd.DataFrame()
            df_final_cancelamentos = pd.DataFrame()


            # DEFININDO DATA DE INICIO DA CAMPANHA E CRIANDO DATAFRAME COM TODAS AS DATAS DE CAMPANHA
            today = dt.date.today()
            yesterday = today - dt.timedelta(days=1)
            last_friday = today - dt.timedelta(days=3)
            comeco_campanha = dt.date(2025, 5, 1)
            lista_dias_faltantes = [dt.date(2025,7,20), dt.date(2025,7,28), dt.date(2025,7,29), 
            dt.date(2025,7,30), dt.date(2025,7,31)]
            date_for_all = pd.date_range(start=comeco_campanha, end=today, freq='D')

        except Exception as e:

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info('Falha ao criar parâmetros de data para filtragem de dataframes.')

        # EXTRAINDO DATAFRAMES E SEGMENTANDO-OS POR EMPRESA / PADRONIZANDO BENEFICIOS
        try:
            extract_inst = Extract()
            df_final_cancelamentos = extract_inst.extract_all_cancelamentos()
            
        
            df_ativ_all_boards = extract_inst.extract_all_ativacoes()
            df_ativ_all_boards['data_ativacao_beneficio'] = pd.to_datetime(df_ativ_all_boards['data_ativacao_beneficio']).dt.date
                        
            df_ativ_all_boards['beneficio'] = df_ativ_all_boards['beneficio'].replace('REPARAÇÃO OU REPOSIÇÃO DO VEÍCULO', 'CASCO (VEÍCULO)').replace('REPARAÇÃO OU REPOSIÇÃO DO (SEMI)REBOQUE', 'CASCO (R/SR)').replace('REPARAÇÃO OU REPOSIÇÃO DO COMPLEMENTO', 'CASCO (COMPLEMENTO)')
            
            df_ativ_viavante = df_ativ_all_boards[df_ativ_all_boards['empresa'] == 'Viavante']
            df_ativ_stcoop = df_ativ_all_boards[df_ativ_all_boards['empresa'] == 'Stcoop']
            df_ativ_segtruck = df_ativ_all_boards[df_ativ_all_boards['empresa'] == 'Segtruck']
            df_ativ_tag = df_ativ_all_boards[df_ativ_all_boards['empresa'] == 'Tag']

            df_conf_all_boards = extract_inst.extract_conf_boards()
            df_conf_all_boards['beneficio'] = df_conf_all_boards['beneficio'].replace('REPARAÇÃO OU REPOSIÇÃO DO VEÍCULO', 'CASCO (VEÍCULO)').replace('REPARAÇÃO OU REPOSIÇÃO DO (SEMI)REBOQUE', 'CASCO (R/SR)').replace('REPARAÇÃO OU REPOSIÇÃO DO COMPLEMENTO', 'CASCO (COMPLEMENTO)')

        except Exception as e:

            logging.info('\n ----------------------------------------------------------------------------------')  
            logging.info(f'Falha ao realizar a segmentação dos dataframes: {e}')



        # SELECIONANDO APENAS AS ATIVAÇÕES CORRESPONDENTES AOS BENEFICIOS 'CASCO' / 'TERCEIRO' POR UM REGEX PADRÃO
        try:

            ids_beneficios_segtruck = [2, 3, 4, 7, 24, 25, 26, 29]
            ids_beneficios_stcoop = [24, 25, 26, 29]
            ids_beneficios_viavante = [40, 41, 42, 45]
            ids_beneficios_tag = [2, 3, 4, 7, 24, 25, 26, 29, 34, 35, 36, 37, 38, 39]

            df_ativ_viavante = df_ativ_viavante.loc[df_ativ_viavante['id_beneficio'].isin(ids_beneficios_viavante)]
            df_ativ_stcoop = df_ativ_stcoop[df_ativ_stcoop['id_beneficio'].isin(ids_beneficios_stcoop)]
            df_ativ_segtruck = df_ativ_segtruck.loc[df_ativ_segtruck['id_beneficio'].isin(ids_beneficios_segtruck)]
            df_ativ_tag = df_ativ_tag.loc[df_ativ_tag['id_beneficio'].isin(ids_beneficios_tag)]

        except Exception as e:

            logging.info('\n ----------------------------------------------------------------------------------')  
            logging.info(f'Falha ao padronizar nomenclaturas referente aos beneficios pré-estabelecidos: {e}')


        # FILTRANDO OS DADOS PELA DATA DE CANCELAMENTO / DATA DE ATUALIZAÇÃO, E ORDENANDO OS DADOS NO DATAFRAME PELAS COLUNAS DE DATA
        try:

            # IDENTIFICANDO CANCELAMENTOS POR MIGRAÇÃO
            #df_final_cancelamentos_integrais = pd.merge(df_final_cancelamentos, df_cancelamentos_dia_anterior_bitrix, on=['conjunto', 'empresa', 'data_cancelamento'], how='left')
            #df_final_cancelamentos.drop(columns=['ID'], inplace=True)
            df_final_cancelamentos_integrais = df_final_cancelamentos
            # FILTRANDO DADOS DE CANCELAMENTOS INTEGRAIS
            #df_final_cancelamentos_integrais['data_cancelamento'] = pd.to_datetime(df_final_cancelamentos_integrais['data_cancelamento']).dt.date
            #df_final_cancelamentos_integrais['data_substituicao'] = pd.to_datetime(df_final_cancelamentos_integrais['data_substituicao']).dt.date
            #if today.weekday() == 0:
            #    df_final_cancelamentos_integrais = df_final_cancelamentos_integrais[
            #        (df_final_cancelamentos_integrais['data_cancelamento'].between(last_friday, today)) |
            #        (df_final_cancelamentos_integrais['data_substituicao'].between(last_friday, today))]
            #else:
            #    df_final_cancelamentos_integrais = df_final_cancelamentos_integrais[
            #        (df_final_cancelamentos_integrais['data_cancelamento'] == yesterday) |
            #        (df_final_cancelamentos_integrais['data_substituicao'] == yesterday)]
                
            # REORDENANDO AS COLUNAS DA BASE DE CANCELAMENTOS INTEGRAIS
            #df_final_cancelamentos_integrais = df_final_cancelamentos_integrais[['placa', 'chassi', 'id_placa', 'id_veiculo', 'id_carroceria', 'conjunto', 'unidade', 'status', 'cliente', 'data_registro', 'data_cancelamento', 'suporte', 'data_extracao', 'empresa', 'data_registro_historico', 'migracao', 'renegociacao', 'data_substituicao']]
            #df_final_cancelamentos_integrais = df_final_cancelamentos_integrais.sort_values(by=['data_cancelamento', 'data_substituicao'], ascending=True)

            
        except Exception as e:

            logging.info('\n ----------------------------------------------------------------------------------')  
            logging.info(f'Falha ao filtrar dados de cancelamentos referente ao dia anterior: {e}')

        # CRIANDO COLUNA DE MIGRAÇÃO (MIGRATION_FROM) E DEFININDO FILTRO DE STATUS PARA TRATAMENTO DE DADOS DA CAMPANHA
        try:
            if not df_ativ_tag.empty:
                df_ativ_tag['migration_from'] = np.nan
            if not df_ativ_viavante.empty:
                df_ativ_viavante['migration_from'] = np.nan
            if not df_ativ_stcoop.empty:
                df_ativ_stcoop['migration_from'] = np.nan
            if not df_ativ_segtruck.empty:
                df_ativ_segtruck['migration_from'] = np.nan
            if not df_ativ_all_boards.empty:
                df_ativ_all_boards['migration_from'] = np.nan

            # CRIANDO LISTA DE VERIFICAÇÃOST DE PLACAS MIGRADAS (STATUS)
            status_filter_list = ['CANCELADO', 'CANCELADA', 'FINALIZADO', 'FINALIZADA', 'NAO RENOVADO']

        except Exception as e:
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Falha na ciração das colunas de migração e definição de filtro de status: {e}')


        # GERANDO DATAFRAME FINAL (TAG)
        '''
            AQUI SÃO FILTRADOS OS DADOS PELA DATA DE ATIVAÇÃO DO BENEFICIO, CASO A DATA SEJA UMA SEGUNDA-FEIRA ELE RETORNA OS DADOS DA SEXTA-FEIRA ANTERIOR ATÉ A SEGUNDA-FEIRA
            CASO CONTRÁRIO, ELE RETORNA OS DADOS DO DIA ANTERIOR.
        '''
        try:
            
            if not df_ativ_tag.empty:
                if today.weekday() == 0:
                    df_final_tg = df_ativ_tag[df_ativ_tag['data_ativacao_beneficio'].between(last_friday, today)]
                else:
                    df_final_tg = df_ativ_tag[df_ativ_tag['data_ativacao_beneficio'] == yesterday]
            else:
                df_final_tg = pd.DataFrame()
                logging.info('\n ----------------------------------------------------------------------------------')
                logging.info(f'Nenhum registro de ativação encontrado na Viavante')

        except Exception as e:

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Falha ao atualizar placas referente a Viavante: {e}')


        # GERANDO DATAFRAME FINAL (VIAVANTE)
        '''
            AQUI SÃO FILTRADOS OS DADOS PELA DATA DE ATIVAÇÃO DO BENEFICIO, CASO A DATA SEJA UMA SEGUNDA-FEIRA ELE RETORNA OS DADOS DA SEXTA-FEIRA ANTERIOR ATÉ A SEGUNDA-FEIRA
            CASO CONTRÁRIO, ELE RETORNA OS DADOS DO DIA ANTERIOR.
        '''
        try:
            
            if not df_ativ_viavante.empty:
                if today.weekday() == 0:
                    df_final_viav = df_ativ_viavante[df_ativ_viavante['data_ativacao_beneficio'].between(last_friday, today)]
                else:
                    df_final_viav = df_ativ_viavante[df_ativ_viavante['data_ativacao_beneficio'] == yesterday]
            else:
                df_final_viav = pd.DataFrame()
                logging.info('\n ----------------------------------------------------------------------------------')
                logging.info(f'Nenhum registro de ativação encontrado na Viavante')

        except Exception as e:

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Falha ao atualizar placas referente a Viavante: {e}')


        # GERANDO DATAFRAME FINAL (STCOOP)
        '''
            AQUI SÃO FILTRADOS OS DADOS PELA DATA DE ATIVAÇÃO DO BENEFICIO, CASO A DATA SEJA UMA SEGUNDA-FEIRA ELE RETORNA OS DADOS DA SEXTA-FEIRA ANTERIOR ATÉ A SEGUNDA-FEIRA
            CASO CONTRÁRIO, ELE RETORNA OS DADOS DO DIA ANTERIOR.
        '''
        try:
            if not df_ativ_stcoop.empty:
                if today.weekday() == 0:
                    df_final_st = df_ativ_stcoop[df_ativ_stcoop['data_ativacao_beneficio'].between(last_friday, today)]
                else:
                    df_final_st = df_ativ_stcoop[df_ativ_stcoop['data_ativacao_beneficio'] == yesterday]
            else:
                df_final_st = pd.DataFrame()
                logging.info('\n ----------------------------------------------------------------------------------')
                logging.info(f'Nenhum registro de ativação encontrado na Stcoop')

        except Exception as e:

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Falha ao atualizar as ativações referente a Stcoop: {e}')

        # GERANDO DATAFRAME FINAL (Segtruck)
        '''
            AQUI SÃO FILTRADOS OS DADOS PELA DATA DE ATIVAÇÃO DO BENEFICIO, CASO A DATA SEJA UMA SEGUNDA-FEIRA ELE RETORNA OS DADOS DA SEXTA-FEIRA ANTERIOR ATÉ A SEGUNDA-FEIRA
            CASO CONTRÁRIO, ELE RETORNA OS DADOS DO DIA ANTERIOR.
        '''
        try:
            if not df_ativ_segtruck.empty:
                if today.weekday() == 0:
                    df_final_seg = df_ativ_segtruck[df_ativ_segtruck['data_ativacao_beneficio'].between(last_friday, today)]
                else:
                    df_final_seg = df_ativ_segtruck[df_ativ_segtruck['data_ativacao_beneficio'] == yesterday]
            else:
                df_final_seg = pd.DataFrame()
                logging.info('\n ----------------------------------------------------------------------------------')
                logging.info(f'Nenhum registro de ativação encontrado na Segtruck')

        except Exception as e:

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Falha ao atualizar as ativações referente a Segtruck: {e}')


        try:
            # PEGANDO DADOS DE ATIVAÇÃO DO DIA ANTERIOR
            if not df_ativ_all_boards.empty:
                df_ativos_menos_ontem = df_ativ_all_boards[~(df_ativ_all_boards['data_ativacao_beneficio'] == yesterday)]
                df_ativos_dia_anterior = df_ativ_all_boards[df_ativ_all_boards['data_ativacao_beneficio'] == yesterday]
                df_ativacoes_dia_anterior_ranking_tratado = Transform.board_status_treatment(df=df_ativos_dia_anterior, df_conf=df_conf_all_boards, status_filter_list=status_filter_list)
                df_ativacoes_atualizado = pd.concat([df_ativos_menos_ontem, df_ativacoes_dia_anterior_ranking_tratado])
                
                logging.info('\n ----------------------------------------------------------------------------------')
                logging.info(f'Número de registros ativos na carteira tratado com os dados do dia anterior.')

        except Exception as e:

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Falha ao incluir registros ativos referente ao dia anterior na contagem . Revise o código: {e}')


        # CRIANDO DATAFRAMES FINAIS: ATIVAÇÕES E CANCELAMENTOS
        try:
            # APLICANDO FUNÇÃO DE TRATAMENTO DE STATUS DAS PLACAS ATIVADAS
            df_final_viavante = Transform.board_status_treatment(df=df_final_viav, df_conf=df_conf_all_boards, status_filter_list=status_filter_list)
            df_final_stcoop = Transform.board_status_treatment(df=df_final_st, df_conf=df_conf_all_boards, status_filter_list=status_filter_list)
            df_final_segtruck = Transform.board_status_treatment(df=df_final_seg, df_conf=df_conf_all_boards, status_filter_list=status_filter_list)
            df_final_tag = Transform.board_status_treatment(df=df_final_tg, df_conf=df_conf_all_boards, status_filter_list=status_filter_list)

            # CONCATENANDO DATAFRAMES DE ATIVAÇÕES DA CAMPANHA
            df_ativacoes_dia_anterior_ranking = pd.concat([df_final_viavante, df_final_stcoop, df_final_segtruck, df_final_tag])
            df_ativacoes_dia_anterior_ranking = df_ativacoes_dia_anterior_ranking.sort_values(by='data_ativacao_beneficio', ascending=True)
            

            # DEFININDO COLUNAS QUE SERÃO UTILIZADAS NOS DATAFRAMES FINAIS

            df_final_ativacoes = df_ativacoes_atualizado[[
                'placa', 'chassi', 'id_placa', 'id_veiculo', 'id_carroceria', 'matricula', 'conjunto', 'unidade', 'consultor', 'status_beneficio', 
                'cliente', 'data_registro', 'data_ativacao_beneficio', 'suporte', 'data_filtro', 'empresa', 'migration_from'
            ]]

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Processo de Concatenação de Dataframes realizado com sucesso!')

        except Exception as e:

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Falha ao unir os dataframes: {e}')

        # TRATANDO NÚMERO DE REGISTROS ATIVOS / RETIRANDO DUPLICATAS
        try:
            df_final_ativacoes = df_final_ativacoes.drop_duplicates(subset='chassi')

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Número de registros ativos tratado e corrigido com sucesso.')

        except Exception as e:

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Falha ao tratar e corrigir número de registros ativos. Revise o código: {e}')

        # TRATANDO DADOS NULOS NOS DATAFRAMES
        try:
            df_final_ativacoes['placa'] = df_final_ativacoes['placa'].fillna('SEM-PLACA')
            df_final_ativacoes['chassi'] = df_final_ativacoes['chassi'].fillna('NULL')
            df_final_ativacoes['id_placa'] = df_final_ativacoes['id_placa'].fillna(0)
            df_final_ativacoes['id_veiculo'] = df_final_ativacoes['id_veiculo'].fillna(0)
            df_final_ativacoes['id_carroceria'] = df_final_ativacoes['id_carroceria'].fillna(0)
            df_final_ativacoes['matricula'] = df_final_ativacoes['matricula'].fillna(0)
            df_final_ativacoes['conjunto'] = df_final_ativacoes['conjunto'].fillna(0)
            df_final_ativacoes['unidade'] = df_final_ativacoes['unidade'].fillna('NULL')
            df_final_ativacoes['consultor'] = df_final_ativacoes['consultor'].fillna('NULL')
            df_final_ativacoes['status_beneficio'] = df_final_ativacoes['status_beneficio'].fillna('NULL')
            df_final_ativacoes['cliente'] = df_final_ativacoes['cliente'].fillna('NULL')
            df_final_ativacoes['data_registro'] = df_final_ativacoes['data_registro'].fillna(pd.Timestamp('1900-01-01'))
            df_final_ativacoes['data_ativacao_beneficio'] = df_final_ativacoes['data_ativacao_beneficio'].fillna(pd.Timestamp('1900-01-01'))
            df_final_ativacoes['suporte'] = df_final_ativacoes['suporte'].fillna('NULL')
            df_final_ativacoes['data_filtro'] = df_final_ativacoes['data_filtro'].fillna(pd.Timestamp('1900-01-01'))
            df_final_ativacoes['empresa'] = df_final_ativacoes['empresa'].fillna('NULL')
            df_final_ativacoes['migration_from'] = df_final_ativacoes['migration_from'].fillna('NULL')

            df_final_cancelamentos_integrais['placa'] = df_final_cancelamentos_integrais['placa'].fillna('SEM-PLACA')
            df_final_cancelamentos_integrais['chassi'] = df_final_cancelamentos_integrais['chassi'].fillna('NULL')
            df_final_cancelamentos_integrais['id_placa'] = df_final_cancelamentos_integrais['id_placa'].fillna(0)
            df_final_cancelamentos_integrais['id_veiculo'] = df_final_cancelamentos_integrais['id_veiculo'].fillna(0)
            df_final_cancelamentos_integrais['id_carroceria'] = df_final_cancelamentos_integrais['id_carroceria'].fillna(0)
            df_final_cancelamentos_integrais['matricula'] = df_final_cancelamentos_integrais['matricula'].fillna(0)
            df_final_cancelamentos_integrais['conjunto'] = df_final_cancelamentos_integrais['conjunto'].fillna(0)
            df_final_cancelamentos_integrais['unidade'] = df_final_cancelamentos_integrais['unidade'].fillna('NULL')
            df_final_cancelamentos_integrais['status'] = df_final_cancelamentos_integrais['status'].fillna('NULL')
            df_final_cancelamentos_integrais['cliente'] = df_final_cancelamentos_integrais['cliente'].fillna('NULL')
            df_final_cancelamentos_integrais['data'] = df_final_cancelamentos_integrais['data'].fillna(pd.Timestamp('1900-01-01'))
            df_final_cancelamentos_integrais['data_cancelamento'] = df_final_cancelamentos_integrais['data_cancelamento'].fillna(pd.Timestamp('1900-01-01'))
            df_final_cancelamentos_integrais['usuario_cancelamento'] = df_final_cancelamentos_integrais['usuario_cancelamento'].fillna('NULL')
            df_final_cancelamentos_integrais['data_filtro'] = df_final_cancelamentos_integrais['data_filtro'].fillna(pd.Timestamp('1900-01-01'))
            df_final_cancelamentos_integrais['empresa'] = df_final_cancelamentos_integrais['empresa'].fillna('NULL')
            df_final_cancelamentos_integrais['migracao'] = df_final_cancelamentos_integrais['migracao'].fillna('NULL')
            df_final_cancelamentos_integrais['renegociacao'] = df_final_cancelamentos_integrais['renegociacao'].fillna('NULL')
            df_final_cancelamentos_integrais['data_substituicao'] = df_final_cancelamentos_integrais['data_substituicao'].fillna(pd.Timestamp('1900-01-01'))

            df_ativacoes_dia_anterior_ranking['placa'] = df_ativacoes_dia_anterior_ranking['placa'].fillna('SEM-PLACA')
            df_ativacoes_dia_anterior_ranking['chassi'] = df_ativacoes_dia_anterior_ranking['chassi'].fillna('NULL')
            df_ativacoes_dia_anterior_ranking['id_placa'] = df_ativacoes_dia_anterior_ranking['id_placa'].fillna(0)
            df_ativacoes_dia_anterior_ranking['id_veiculo'] = df_ativacoes_dia_anterior_ranking['id_veiculo'].fillna(0)
            df_ativacoes_dia_anterior_ranking['id_carroceria'] = df_ativacoes_dia_anterior_ranking['id_carroceria'].fillna(0)
            df_ativacoes_dia_anterior_ranking['matricula'] = df_ativacoes_dia_anterior_ranking['matricula'].fillna(0)
            df_ativacoes_dia_anterior_ranking['conjunto'] = df_ativacoes_dia_anterior_ranking['conjunto'].fillna(0)
            df_ativacoes_dia_anterior_ranking['unidade'] = df_ativacoes_dia_anterior_ranking['unidade'].fillna('NULL')
            df_ativacoes_dia_anterior_ranking['consultor'] = df_ativacoes_dia_anterior_ranking['consultor'].fillna('NULL')
            df_ativacoes_dia_anterior_ranking['status'] = df_ativacoes_dia_anterior_ranking['status'].fillna('NULL')
            df_ativacoes_dia_anterior_ranking['cliente'] = df_ativacoes_dia_anterior_ranking['cliente'].fillna('NULL')
            df_ativacoes_dia_anterior_ranking['data'] = df_ativacoes_dia_anterior_ranking['data'].fillna(pd.Timestamp('1900-01-01'))
            df_ativacoes_dia_anterior_ranking['data_ativacao'] = df_ativacoes_dia_anterior_ranking['data_ativacao'].fillna(pd.Timestamp('1900-01-01'))
            df_ativacoes_dia_anterior_ranking['suporte'] = df_ativacoes_dia_anterior_ranking['suporte'].fillna('NULL')
            df_ativacoes_dia_anterior_ranking['data_filtro'] = df_ativacoes_dia_anterior_ranking['data_filtro'].fillna(pd.Timestamp('1900-01-01'))
            df_ativacoes_dia_anterior_ranking['empresa'] = df_ativacoes_dia_anterior_ranking['empresa'].fillna('NULL')
            df_ativacoes_dia_anterior_ranking['migration_from'] = df_ativacoes_dia_anterior_ranking['migration_from'].fillna('NULL')
            df_ativacoes_dia_anterior_ranking['coverage'] = df_ativacoes_dia_anterior_ranking['coverage'].fillna(0)
            df_ativacoes_dia_anterior_ranking['beneficio'] = df_ativacoes_dia_anterior_ranking['beneficio'].fillna('NULL')
            df_ativacoes_dia_anterior_ranking['categoria'] = df_ativacoes_dia_anterior_ranking['categoria'].fillna('NULL')
            df_ativacoes_dia_anterior_ranking['tipo_categoria'] = df_ativacoes_dia_anterior_ranking['tipo_categoria'].fillna('NULL')
            df_ativacoes_dia_anterior_ranking['data_ativacao_beneficio'] = df_ativacoes_dia_anterior_ranking['data_ativacao_beneficio'].fillna(pd.Timestamp('1900-01-01'))
            df_ativacoes_dia_anterior_ranking['status_beneficio'] = df_ativacoes_dia_anterior_ranking['status_beneficio'].fillna('NULL')
            df_ativacoes_dia_anterior_ranking['data_atualizacao'] = df_ativacoes_dia_anterior_ranking['data_atualizacao'].fillna(pd.Timestamp('1900-01-01'))
            df_ativacoes_dia_anterior_ranking['microfranquia'] = df_ativacoes_dia_anterior_ranking['microfranquia'].fillna('NULL')
            df_ativacoes_dia_anterior_ranking['id_beneficio'] = df_ativacoes_dia_anterior_ranking['id_beneficio'].fillna(0)
        

            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info('\n Processo de Transformacao de Dados concluido com sucesso!')

        except Exception as e:
            logging.info('\n ----------------------------------------------------------------------------------')
            logging.info(f'Falha ao realizar tratamento de dados: {e}')
     

        return df_final_ativacoes, df_final_cancelamentos_integrais, df_ativacoes_dia_anterior_ranking
    
transform = Transform()

df_final_ativacoes, df_final_cancelamentos_integrais, df_ativacoes_dia_anterior_ranking = transform.transforming_files()

        

2025-08-26 09:50:35,562 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 09:50:37,911 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 09:50:40,553 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 09:50:43,188 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 09:50:45,327 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 09:50:47,303 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 09:50:49,180 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 09:50:51,140 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 09:50:54,183 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 09:50:56,094 - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2025-08-26 09:50:58,

In [3]:
df_final_ativacoes_ativos = df_final_ativacoes[df_final_ativacoes['status_beneficio']=='ATIVO']
df_final_ativacoes_ativos.shape[0]

33088

# LOAD

In [ ]:
# IMPORTANDO MÓDULOS E PACOTES
import logging
import openpyxl
import shutil
import os
import traceback
import pandas as pd
import logging



file_path = r"C:\Users\raphael.almeida\Documents\Processos\placas_acompanhamento\template\placas_movimentacoes.xlsx"
destination_dir = r"C:\Users\raphael.almeida\OneDrive - Grupo Unus\analise de dados - Arquivos em excel\Relatório de Ativações Placas"
destination_path = os.path.join(destination_dir, os.path.basename(file_path))

with pd.ExcelWriter(destination_path, engine='openpyxl') as writer:
    df_final_ativacoes.to_excel(writer, index=False, sheet_name='ATIVAÇÕES')
    df_final_cancelamentos_integrais.to_excel(writer, index=False, sheet_name='CANCELAMENTOS')



logging.info('\n ----------------------------------------------------------------------------------')
logging.info('\n Processo de Carregamento de Dados concluido com sucesso!')



    # extract_instance = Extract() #Fazer isso quando a class contiver instâncias 

    # df_ativacoes = extract_instance.extract_all_ativacoes() #salvando em variáveis os métodos da instância de classe
    # df_cancelamentos = extract_instance.extract_all_cancelamentos() 
    # df_conferencia = extract_instance.extract_conf_boards()
   
    # Criando uma instância da classe Transform, pois não é uma classe estática (usa self)
    # transform_instance = Transform()  # Passando um dicionário vazio como config

    # Chamando o método transforming_files da classe Transform, que retorna os DataFrames processados    
    # df_final_ativacoes, df_cancelamentos = transform_instance.transforming_files(df_ativacoes, df_cancelamentos, df_conferencia)

    # Criando uma instância da classe Load_camp_rank_ativ, pois ela não é estática (usa self)
    # load_instance = Load_camp_rank_ativ()

    # Chamando o método load_files da instância da classe Load_camp_rank_ativ, passando os DataFrames processados
    # load_instance.load_files(df_final_ativacoes, df_cancelamentos)







    


2025-08-26 09:55:58,998 - INFO - 
 ----------------------------------------------------------------------------------
2025-08-26 09:55:59,000 - INFO - 
 Processo de Carregamento de Dados concluido com sucesso!
